In [1]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import torch
import torchvision.models as models
import torch.nn as nn
from torch.nn.functional import sigmoid


c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset_path = "the_wildfire_dataset"
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
])
# Load datasets
train_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'train'), transform=transform)
val_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'val'), transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(dataset_path, 'test'), transform=transform)


In [7]:
device = "cuda"
# Define the same model structure as was used for training
model = models.resnet101(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, 1)  # Assuming it's the modified ResNet-101 for binary classification

# Load the model weights
model_path = 'trained_resnet101.pth'  # Replace with your .pth file path
model.load_state_dict(torch.load(model_path))
model.to(device)
# If you're planning to use the model for inference, switch to evaluation mode
model.eval()

c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [8]:
# Data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataset_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [9]:
# Validation
model.eval()
running_corrects = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_dataset_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = sigmoid(model(inputs).squeeze())  # Apply sigmoid
        predicted = outputs > 0.5  # Threshold to get binary class
        total += labels.size(0)
        running_corrects += (predicted == labels).sum().item()

c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\torch\nn\functional.py:1967: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (101859328 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (96631920 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
c:\Users\Blues\miniconda3\envs\stable\lib\site-packages\PIL\Image.py:3167: DecompressionBombWarning: Image size (94487082 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


In [10]:
# Log validation accuracy
val_accuracy = 100 * running_corrects / total
print(f'Validation Accuracy: {val_accuracy:.2f}%')

Validation Accuracy: 77.56%
